In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllxiii.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllxiii.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllxiii.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para extraer la entidad de la página de un diputado
def extraer_entidad(soup):
    try:
        entidad_texto = soup.find(text=lambda text: "Entidad:" in text).find_next()
        entidad = entidad_texto.get_text(strip=True) if entidad_texto else "No disponible"
        return entidad.split("|")[0].strip()
    except AttributeError:
        return "No disponible"


# Función para obtener la información completa de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    
    # Corrección en la extracción del nombre del diputado
    elemento_nombre = soup_diputado.find('strong', text=lambda t: "Dip." in t)
    nombre = elemento_nombre.get_text(strip=True) if elemento_nombre else 'Nombre no disponible'
    
    entidad = extraer_entidad(soup_diputado)
    
    # Las siguientes líneas se mantienen sin cambios
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    
    return {
        'nombre': nombre,
        'entidad': entidad,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                diputado_info = future.result()
                if diputado_info:
                    datos_diputados.append(diputado_info)
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

                                   nombre       entidad  \
0         Dip. Erubiel Lorenzo Alonso Que       Tabasco   
1    Dip. María Guadalupe Alcántara Rojas        México   
2          Dip. Antonio Tarek Abdala Saad      Veracruz   
3       Dip. Edith Anabel Alvarado Varela      Tlaxcala   
4               Dip. Fidel Almanza Monroy        México   
..                                    ...           ...   
495           Dip. Javier Guerrero García      Coahuila   
496              Dip. Adriana Sarur Torre      Veracruz   
497          Dip. José Luis Toledo Medina  Quintana Roo   
498          Dip. Francisco Martínez Neri        Oaxaca   
499            Dip. Edgar Spinoso Carrera      Veracruz   

                                             educacion  \
0                                                   []   
1    [{'grado': 'Licenciatura', 'campo': 'Contadurí...   
2    [{'grado': 'Licenciatura', 'campo': 'Prensa Es...   
3    [{'grado': 'Maestría', 'campo': 'Políticas Púb...   
4

In [3]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados_1.4.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados_1.4.csv
